In [1]:
import geopandas as gpd
import pandas as pd
import plotly.express as px
import numpy as np
import base64, io

# create an encocded image of graph...
# change to generate graph you want
def b64image(vals=np.random.randint(1, 25, 5)):
    fig = px.bar(
        pd.DataFrame({"y": vals}).pipe(
            lambda d: d.assign(category=d.index.astype(str))
        ),
        y="y",
        color="category",
    ).update_layout(
        showlegend=False,
        xaxis_visible=False,
        yaxis_visible=False,
        bargap=0,
        margin={"l": 0, "r": 0, "t": 0, "b": 0},
        autosize=False,
        height=100,
        width=100,
        paper_bgcolor="rgba(0,0,0,0)",
        plot_bgcolor="rgba(0,0,0,0)",
    )

    b = io.BytesIO(fig.to_image(format="png"))
    b64 = base64.b64encode(b.getvalue())
    return "data:image/png;base64," + b64.decode("utf-8"), fig

# get some geometry
world = gpd.read_file("/Users/jm/Code/geopandas/geopandas/datasets/naturalearth_lowres/naturalearth_lowres.shp")
# let's just work with a bounded version of europe
eur = world.loc[
    lambda d: d["continent"].eq("Europe")
    & ~d["iso_a3"].isin(["RUS", "NOR", "FRA", "ISL"])
]


px.choropleth_mapbox(
    eur,
    geojson=eur.__geo_interface__,
    locations="iso_a3",
    featureidkey="properties.iso_a3",
    color_discrete_sequence=["lightgrey"],
).update_layout(
    margin={"l": 0, "r": 0, "t": 0, "b": 0},
    showlegend=False,
    mapbox_style="carto-positron",
    mapbox_center={
        "lon": eur.unary_union.centroid.x,
        "lat": eur.unary_union.centroid.y,
    },
    mapbox_zoom=3,
    # add a plotly graph per country...
    mapbox_layers=[
        {
            "sourcetype": "image",
            # no data provided, use random values for each country
            "source": b64image(vals=np.random.randint(1, 25, 5))[0],
            # https://plotly.com/python/reference/layout/mapbox/#layout-mapbox-layers-items-layer-coordinates
            # a few hops to get 4 cordinate pairs to meet mapbox requirement
            "coordinates": [
                list(p) for p in r.geometry.centroid.buffer(1.1).envelope.exterior.coords
            ][0:-1][::-1],
        }
        for i, r in eur.iterrows()
    ],
)

/var/folders/8j/bf4mjlj94czf88p79v8gb24w0000gp/T/ipykernel_3292/2278024872.py:53: DeprecationWarning:

The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.

/var/folders/8j/bf4mjlj94czf88p79v8gb24w0000gp/T/ipykernel_3292/2278024872.py:54: DeprecationWarning:

The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.

